Basic Imports and Device Setup

In [ ]:
# Standard Libraries
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# PyTorch Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [ ]:
# Scikit-learn for Preprocessing and Metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.manifold import TSNE
from scipy.stats import gaussian_kde

In [ ]:
import os
import requests
import pickle

In [ ]:
# Visualization Settings
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

In [ ]:
# Reproducibility: Set random seed for consistent results
SEED = 42
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

In [ ]:
# Device Setup: Enable GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Optional: Display CUDA device information
if device.type == "cuda":
    print(f"CUDA Device Name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Device Count: {torch.cuda.device_count()}")
    print(f"CUDA Available: {torch.cuda.is_available()}")

In [ ]:
# Utility function to check memory usage (if needed during training)
def print_memory_usage():
    if device.type == "cuda":
        allocated = torch.cuda.memory_allocated() / 1024**2
        reserved = torch.cuda.memory_reserved() / 1024**2
        print(f"Memory Allocated: {allocated:.2f} MB")
        print(f"Memory Reserved: {reserved:.2f} MB")

In [ ]:
# Confirming imports and environment setup
print("Basic imports and device setup complete.")